In [1]:
# add the root to the path
import sys
import pandas as pd
sys.path.append('/Users/pmassaro/Documents/Repos/JobSeeker')
from sqlalchemy import text
from jobseeker.database import DatabaseManager
from jobseeker.llm import ModelNames
from jobseeker.llm.cv_data_extractor import CVLLMExtractor
from jobseeker.llm.job_description_extractor import JobDescriptionLLMExtractor
from jobseeker.llm.cv_builder import CVBuilder
from jobseeker.llm.requirements_qualification_comparator import RequirementsQualificationsComparator
from jobseeker.llm.cover_letter_builder import CoverLetterBuilder


db = DatabaseManager()
user_id = 1


cv_extractor = CVLLMExtractor(model_name=ModelNames.GPT4_TURBO,temperature=0)
cv_extractor.extract_cv_and_write_to_db(user_id=user_id)


2024-04-26 10:19:34,475 - CVExtractor - INFO - Extracting CV data for user 1
2024-04-26 10:20:46,598 - CVExtractor - INFO - Extracted data from text: 
 Tokens Used: 3087
	Prompt Tokens: 1677
	Completion Tokens: 1410
Successful Requests: 1
Total Cost (USD): $0.05907
2024-04-26 10:20:46,612 - CVExtractor - INFO - Successfully extracted CV data for user 1


1

In [64]:
# create a query selecting all the job postings
query = text("""
    SELECT 
        *
    FROM job_postings JP
             
    where 
        lower(company) like '%writer%' 
        and (lower(title) like '%ai%'  
          or lower(title) like '%scientist%'
     )
    --and id in (3746695448,3746693430,3814013833,3774906497,3854675033,3746692698)

""" )
session = db.get_session()
job_postings= session.execute(query)
job_postings = pd.DataFrame(job_postings.fetchall(), columns=job_postings.keys())
session.close()
job_ids = job_postings['id'].tolist()
job_postings

,id,title,seniority_level,employment_type,job_description,company,company_url,industries,job_functions,institution_id,job_salary_range_max,job_salary_range_min,job_poster_profile_url,job_poster_name,skills,date_created,job_posting_summary,date_updated
0,3725730599,AI Engineer,Mid-Senior level,Full-time,✍🏽 About Writer\n\n\nAI your people will love....,Writer,https://www.linkedin.com/company/getwriter,[Software Development],[Engineering Information Technology],None,None,None,None,None,None,2024-04-25 15:32:11.784953,None,2024-04-25 15:32:11.784953


# Create job description and CV summaries using LLMs

In [65]:
job_posting_parser = JobDescriptionLLMExtractor(model_name=ModelNames.GPT4_TURBO,temperature=0)
job_posting_parser.update_job_postings(job_ids=job_ids)

cv_extractor = CVLLMExtractor(model_name=ModelNames.GPT4_TURBO,temperature=0)
cv_extractor.extract_cv_and_write_to_db(user_id=user_id)

comparator = RequirementsQualificationsComparator(model_name=ModelNames.GPT4_TURBO,
                                                  temperature=0.55,
                                                  user_id=user_id)
comparator.compare_job_postings_with_user(job_ids=job_ids,replace_previous_comparison_flag=True)


2024-04-25 11:32:35,370 - JobDescriptionExtractor - INFO - Extracting job posting 3725730599...
2024-04-25 11:33:13,503 - JobDescriptionExtractor - INFO - Extracted data from text: 
 Tokens Used: 3387
	Prompt Tokens: 2717
	Completion Tokens: 670
Successful Requests: 1
Total Cost (USD): $0.04727
2024-04-25 11:33:13,526 - JobDescriptionExtractor - INFO - Job posting 3725730599 updated successfully.
2024-04-25 11:33:13,581 - CVExtractor - INFO - Extracting CV data for user 1
2024-04-25 11:33:13,610 - CVExtractor - INFO - User 1 already has a CV summary. Set replace_existing_summary to True to overwrite.
2024-04-25 11:33:13,673 - RequirementsQualificationsComparator - INFO - Comparing requirements and qualifications for user 1 and job posting 3725730599
2024-04-25 11:33:58,085 - RequirementsQualificationsComparator - INFO - Extracted data from text: 
 Tokens Used: 3446
	Prompt Tokens: 2740
	Completion Tokens: 706
Successful Requests: 1
Total Cost (USD): $0.04858
2024-04-25 11:33:58,131 - D

Now, query the DB and get the relevant info

# Build CV

In [66]:
cv_builder = CVBuilder(model_name=ModelNames.GPT4_TURBO,temperature=0.6,user_id=user_id)
cv_builder.build(job_ids=job_ids)


2024-04-25 11:44:46,943 - CVBuilder - INFO - Job 3725730599
2024-04-25 11:46:03,165 - CVBuilder - INFO - Built CV for user 1 and 3725730599 
 Tokens Used: 4374
	Prompt Tokens: 3282
	Completion Tokens: 1092
Successful Requests: 1
Total Cost (USD): $0.06558
2024-04-25 11:46:04,076 - CVBuilder - INFO - PDF created successfully: /Users/pmassaro/Documents/Repos/JobSeeker/jobseeker/media/1/Writer_AIEngineer_CV.pdf
2024-04-25 11:46:08,086 - CVBuilder - INFO - Successfully processed job posting 3725730599 with result: None


# Build Cover Letter

In [55]:
cover_letter_builder = CoverLetterBuilder(model_name=ModelNames.GPT4_TURBO,temperature=0.5,user_id=user_id)
cover_letter_builder.build(job_ids=job_ids)

                       

2024-04-25 11:14:32,022 - CVBuilder - INFO - Job 3746695448
2024-04-25 11:14:32,022 - CVBuilder - INFO - Job 3746693430
2024-04-25 11:14:32,022 - CVBuilder - INFO - Job 3814013833
2024-04-25 11:14:32,022 - CVBuilder - INFO - Job 3774906497
2024-04-25 11:14:32,022 - CVBuilder - INFO - Job 3854675033
2024-04-25 11:14:47,866 - CVBuilder - INFO - Built Cover Letter for user 1 and 3746695448: 
 Tokens Used: 4542
	Prompt Tokens: 4267
	Completion Tokens: 275
Successful Requests: 1
Total Cost (USD): $0.05092000000000001
2024-04-25 11:14:48,319 - CVBuilder - INFO - PDF created successfully: /Users/pmassaro/Documents/Repos/JobSeeker/jobseeker/media/1/Figma_MLAIEngineerAppliedResearch_cover_letter.pdf
2024-04-25 11:14:48,338 - CVBuilder - INFO - Built Cover Letter for user 1 and 3774906497: 
 Tokens Used: 4463
	Prompt Tokens: 4181
	Completion Tokens: 282
Successful Requests: 1
Total Cost (USD): $0.050269999999999995
2024-04-25 11:14:48,722 - CVBuilder - INFO - PDF created successfully: /Users/pma